# Isolating Context

*Isolating context involves splitting it up to help an agent perform a task.*

## Multi-Agent

One of the most popular and intuitive ways to isolate context is to split it across sub-agents. A motivation for the OpenAI [Swarm](https://github.com/openai/swarm) library was “[separation of concerns](https://openai.github.io/openai-agents-python/ref/agent/)”, where a team of agents can handle sub-tasks. Each agent has a specific set of tools, instructions, and its own context window.

Anthropic’s [multi-agent researcher](https://www.anthropic.com/engineering/built-multi-agent-research-system) makes a clear case for the benefit of this: many agents with isolated contexts outperformed single-agent by 90.2%, largely because each subagent context window can be allocated to a more narrow sub-task. As the blog said:

> [Subagents operate] in parallel with their own context windows, exploring different aspects of the question simultaneously. 

Of course, the challenge with multi-agent include token use (e.g., [15× more tokens](https://www.anthropic.com/engineering/built-multi-agent-research-system) than chat), the need for careful [prompt engineering](https://www.anthropic.com/engineering/built-multi-agent-research-system) to plan sub-agent work, and coordination of sub-agents.

### Multi-Agent in LangGraph

LangGraph supports multi-agent systems. A popular and intuitive way to implement this is the [supervisor](https://github.com/langchain-ai/langgraph-supervisor-py) architecture, which is what is used in Anthropic's [multi-agent researcher](https://www.anthropic.com/engineering/built-multi-agent-research-system). This allows the supervisor to delegate tasks to sub-agents, each with their own context window.

In [2]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

# Augment the LLM with tools
from langchain.chat_models import init_chat_model
llm = init_chat_model("anthropic:claude-sonnet-4-20250514", temperature=0)

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})

Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


In [3]:
#TODO: Lets move this to a utils file and import it 

# Use Rich to display messages with enhanced formatting
from rich.console import Console
from rich.panel import Panel
import json

console = Console()

def format_message_content(message):
    """Convert message content to displayable string"""
    if isinstance(message.content, str):
        return message.content
    elif isinstance(message.content, list):
        # Handle complex content like tool calls
        parts = []
        for item in message.content:
            if item.get('type') == 'text':
                parts.append(item['text'])
            elif item.get('type') == 'tool_use':
                parts.append(f"\n🔧 Tool Call: {item['name']}")
                parts.append(f"   Args: {json.dumps(item['input'], indent=2)}")
        return "\n".join(parts)
    else:
        return str(message.content)

def format_messages(messages):

    for m in messages:
        msg_type = m.__class__.__name__.replace('Message', '')
        content = format_message_content(m)

        if msg_type == 'Human':
            console.print(Panel(content, title="🧑 Human", border_style="blue"))
        elif msg_type == 'Ai':
            console.print(Panel(content, title="🤖 Assistant", border_style="green"))
        elif msg_type == 'Tool':
            console.print(Panel(content, title="🔧 Tool Output", border_style="yellow"))
        else:
            console.print(Panel(content, title=f"📝 {msg_type}", border_style="white"))

format_messages(result['messages'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ what's the combined headcount of the FAANG companies in 2024?                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you find the combined headcount of the FAANG companies (Facebook/Meta, Apple, Amazon, Netflix,        │
│ Google/Alphabet) for 2024. This requires researching current employment data for these major tech companies.    │
│                                                                                                                 │
│ 🔧 Tool Call: transfer_to_research_expert                                                                       │
│    Args: {}                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Successfully transferred to research_expert                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Based on my research, here are the 2024 employee headcounts for the FAANG companies:                            │
│                                                                                                                 │
│ **Individual Company Headcounts (2024):**                                                                       │
│ - **Meta (Facebook)**: 67,317 employees                                                                         │
│ - **Apple**: 164,000 employees                                                                                  │
│ - **Amazon**: 1,551,000 employees                                                                               │
│ - **Netflix**: 14,000 employees                                                                                 │
│ - **Google (Alphabet)**: 181,269 employees                                                                      │
│                                                                                                                 │
│ **Combined FAANG Headcount**: 1,977,586 employees                                                               │
│                                                                                                                 │
│ Amazon represents by far the largest portion of this total workforce, accounting for approximately 78% of the   │
│ combined FAANG headcount. This is largely due to Amazon's massive logistics, warehousing, and retail operations │
│ that require significant physical workforce beyond just tech roles.                                             │
│                                                                                                                 │
│ It's worth noting that these numbers represent the most recent available data for 2024, and employee counts at  │
│ these companies can fluctuate due to hiring, layoffs, and business changes throughout the year.                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Transferring back to supervisor                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Successfully transferred back to supervisor                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Learn more

* TODO: Add notes on multi-agent [SWARM](https://github.com/langchain-ai/langgraph-swarm-py)
* [See](https://www.youtube.com/watch?v=4nZl32FwU-o) [these](https://www.youtube.com/watch?v=JeyDrn1dSUQ) [videos](https://www.youtube.com/watch?v=B_0TNuYi56w) for more detail on on multi-agent systems.

## Sandboxed Environment

HuggingFace’s [deep researcher](https://huggingface.co/blog/open-deep-research#:~:text=From%20building%20,it%20can%20still%20use%20it) shows another interesting example of context isolation. Most agents use [tool calling APIs](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/overview), which return JSON objects (tool arguments) that can be passed to tools (e.g., a search API) to get tool feedback (e.g., search results). HuggingFace uses a [CodeAgent](https://huggingface.co/papers/2402.01030), which outputs code to invoke tools. The code then runs in a [sandbox](https://e2b.dev/). Selected context (e.g., return values) from code execution is then passed back to the LLM.

This allows context to be isolated in the environment, outside of the LLM context window. Hugging Face noted that this is a great way to isolate token-heavy objects from the LLM:

> [Code Agents allow for] a better handling of state … Need to store this image / audio / other for later use? No problem, just assign it as a variable in your state and you [use it later].

### Sandboxed Environment in LangGraph

It's pretty easy to use Sandboxes with LangGraph agents. [LangChain Sandbox](https://github.com/langchain-ai/langchain-sandbox) provides a secure environment for executing untrusted Python code. It leverages Pyodide (Python compiled to WebAssembly) to run Python code in a sandboxed environment. This can simply be used as a tool in a LangGraph agent.

> NOTE: Install Deno (required): https://docs.deno.com/runtime/getting_started/installation/

In [5]:
from langchain_sandbox import PyodideSandboxTool
tool = PyodideSandboxTool()
result = await tool.ainvoke("print('Hello, world!')")

In [6]:
from langchain_sandbox import PyodideSandboxTool
tool = PyodideSandboxTool()
result = await tool.ainvoke("print('Hello, world!')")

In [7]:
from langgraph.prebuilt import create_react_agent
from langchain_sandbox import PyodideSandboxTool

tool = PyodideSandboxTool(
    # Allow Pyodide to install python packages that
    # might be required.
    allow_net=True
)
agent = create_react_agent(
    "anthropic:claude-3-7-sonnet-latest",
    tools=[tool],
)
result = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
)

format_messages(result['messages'])


╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ what's 5 + 7?                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I can calculate 5 + 7 for you using the Python code sandbox.                                                    │
│                                                                                                                 │
│ 🔧 Tool Call: python_code_sandbox                                                                               │
│    Args: {                                                                                                      │
│   "code": "print(5 + 7)"                                                                                        │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Didn't find package micropip-0.9.0-py3-none-any.whl locally, attempting to load from                            │
│ https://cdn.jsdelivr.net/pyodide/v0.27.7/full/Didn't find package packaging-24.2-py3-none-any.whl locally,      │
│ attempting to load from https://cdn.jsdelivr.net/pyodide/v0.27.7/full/Package micropip-0.9.0-py3-none-any.whl   │
│ loaded from https://cdn.jsdelivr.net/pyodide/v0.27.7/full/, caching the wheel in node_modules for future        │
│ use.Package packaging-24.2-py3-none-any.whl loaded from https://cdn.jsdelivr.net/pyodide/v0.27.7/full/, caching │
│ the wheel in node_modules for future use.12                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The answer to 5 + 7 is 12.                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### State 

An agent’s runtime state object can also be a great way to isolate context. This can serve the same purpose as sandboxing. A state object can be designed with a schema (e.g., a Pydantic model) that has various fields that context can be written to. One field of the schema (e.g., messages) can be exposed to the LLM at each turn of the agent, but the schema can isolate information in other fields for more selective use. 

### State Isolation in LangGraph

LangGraph is designed around a [state](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) object, allowing you to design a state schema and access different fields of that schema across trajectory of your agent. For example, you can easily store context from tool calls in certain fields of your state object, isolating from the LLM until that context is required. In these notebooks, you've seen numerous example of this.
